In [ ]:
import torch
import random
from transformers import LlamaForCausalLM, LlamaTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy import stats

# Block 1: Dataset Generation
def generate_dataset():
    np.random.seed(42)
    data = pd.DataFrame({
        'Feature_A': np.random.rand(100) * 100,
        'Feature_B': np.random.rand(100) * 100 + 10,
        'Feature_C': np.random.rand(100) * 100 - 5,
        'Target': np.random.choice([0, 1], 100)
    })
    data['Feature_B'] += data['Feature_A'] * 0.8  # Correlation
    return data

# Block 2: PyTorch Model Definition and Training
class SimpleNN(torch.nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = torch.nn.Linear(input_dim, 64)
        self.layer2 = torch.nn.Linear(64, 32)
        self.output = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return torch.sigmoid(self.output(x))

# Block 3: Optimizers and Loss Functions
def get_optimizer_list(model, lr=0.001):
    optimizers = {
        "Adam": torch.optim.Adam(model.parameters(), lr=lr),
        "SGD": torch.optim.SGD(model.parameters(), lr=lr),
        "RMSprop": torch.optim.RMSprop(model.parameters(), lr=lr),
        "Adagrad": torch.optim.Adagrad(model.parameters(), lr=lr),
    }
    return optimizers

def get_loss_functions():
    return {
        "Binary Cross Entropy": torch.nn.BCELoss(),
        "Mean Squared Error": torch.nn.MSELoss(),
        "Cross Entropy Loss": torch.nn.CrossEntropyLoss(),
    }

# Block 4: RL Agent Actions
def agent_action(data, action, params, model):
    feedback = ""
    reward = 0  # Initialize reward

    if action == "choose_optimizer":
        optimizers = get_optimizer_list(model, lr=params['learning_rate'])
        chosen_optimizer_name = random.choice(list(optimizers.keys()))
        chosen_optimizer = optimizers[chosen_optimizer_name]
        feedback = f"Chose {chosen_optimizer_name} optimizer."
        reward = 5  # Reward for exploring optimizers
    elif action == "choose_loss_function":
        loss_functions = get_loss_functions()
        chosen_loss_name = random.choice(list(loss_functions.keys()))
        chosen_loss_function = loss_functions[chosen_loss_name]
        feedback = f"Chose {chosen_loss_name} loss function."
        reward = 5  # Reward for exploring loss functions
    elif action == "apply_pca":
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data.drop(columns=['Target']))
        pca = PCA(n_components=2)
        reduced_data = pca.fit_transform(scaled_data)
        data = pd.DataFrame(reduced_data, columns=['PC1', 'PC2'])
        data['Target'] = data.index.map(data['Target'])
        feedback = "Applied PCA, reduced features to 2 principal components."
        reward = 5  # Reward for reducing dimensionality
    elif action == "adjust_split":
        params['test_size'] = random.uniform(0.1, 0.4)
        feedback = f"Adjusted test/train split to {params['test_size']:.2f} for better evaluation."
        reward = 3  # Small reward for exploration
    return data, feedback, params, reward

# Block 5: LLM Query (Using Hugging Face Llama)
def query_llm(prompt):
    model_name = "huggingface/llama-7b"  # Llama model with 7 billion parameters
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    model = LlamaForCausalLM.from_pretrained(model_name)

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Block 6: Training Model with Optimizer and Loss Function Choices
def train_model(data, test_size=0.2, learning_rate=0.001, epsilon=1e-4, optimizer=None, loss_fn=None):
    X = data.drop(columns=['Target']).values
    y = data['Target'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

    model = SimpleNN(X_train.shape[1])

    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    if loss_fn is None:
        loss_fn = torch.nn.BCELoss()

    # Training loop
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(X_train)
        loss = loss_fn(output, y_train)
        loss.backward()
        optimizer.step()

        if loss.item() < epsilon:
            break

    # Evaluate the model
    with torch.no_grad():
        output = model(X_test)
        predicted = (output > 0.5).float()
        accuracy = (predicted == y_test).float().mean().item()

    return accuracy

# Block 7: RL Loop
def rl_loop():
    data = generate_dataset()
    print("Initial Dataset:\n", data.head())

    actions = ["choose_optimizer", "choose_loss_function", "apply_pca", "adjust_split"]
    params = {'test_size': 0.2, 'learning_rate': 0.001, 'epsilon': 1e-4}

    reward_history = []
    accuracy_history = []

    for step in range(10):  # Simulate 10 RL steps
        action = random.choice(actions)
        print(f"Step {step + 1}: Agent chose action: {action}")
        
        data, feedback, params, reward = agent_action(data, action, params, model=None)

        print("Action Feedback:", feedback)
        llm_response = query_llm(f"Explain the impact of this action: {feedback}")
        print("LLM Response:", llm_response)

        optimizer = None
        loss_fn = None
        if "optimizer" in action:
            optimizer = get_optimizer_list(None, lr=params['learning_rate'])[random.choice(list(get_optimizer_list(None).keys()))]
        if "loss_function" in action:
            loss_fn = get_loss_functions()[random.choice(list(get_loss_functions().keys()))]

        accuracy = train_model(data, test_size=params['test_size'], learning_rate=params['learning_rate'], epsilon=params['epsilon'], optimizer=optimizer, loss_fn=loss_fn)
        print(f"Model Accuracy after this action: {accuracy:.2f}")

        # Log rewards and accuracy
        reward_history.append(reward)
        accuracy_history.append(accuracy)

        print(f"Reward for this step: {reward}")
        print("-" * 50)

    # Visualization
    visualize_results(reward_history, accuracy_history)

# Block 8: Visualization
def visualize_results(reward_history, accuracy_history):
    steps = list(range(1, len(reward_history) + 1))

    plt.figure(figsize=(12, 6))

    # Reward History
    plt.subplot(1, 2, 1)
    plt.plot(steps, reward_history, marker='o', color='blue')
    plt.title("Reward History")
    plt.xlabel("Steps")
    plt.ylabel("Reward")
    plt.grid(True)

    # Accuracy History
    plt.subplot(1, 2, 2)
    plt.plot(steps, accuracy_history, marker='o', color='green')
    plt.title("Accuracy History")
    plt.xlabel("Steps")
    plt.ylabel("Accuracy")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Main Execution
if __name__ == "__main__":
    rl_loop()


In [ ]:
import openai
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# LLM Query Function
def query_llm(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a helpful RPG game assistant specializing in Machine Learning."},
                  {"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message['content']

# Step 1: Generate Corrupted Data
def generate_corrupted_data():
    data = {
        'Temperature': [30, None, 25, 35, None, 28, 40],
        'Humidity': [70, 65, None, 80, 75, None, 60],
        'Rain': [1, 0, 1, None, 0, 1, 0]
    }
    return pd.DataFrame(data)

# Step 2: Preprocess the Data
def clean_data(df):
    df = df.fillna(df.mean())  # Fill missing values with mean
    return df

# Step 3: Train Logistic Regression
def train_model(df):
    X = df[['Temperature', 'Humidity']]
    y = df['Rain']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return model, accuracy, report

# Main RPG Game Flow
def play_game():
    print(query_llm("Start the RPG game and introduce a scenario involving a corrupted weather scroll."))

    data = generate_corrupted_data()
    print("Corrupted Data:\n", data)

    user_action = input("What would you like to do? (Options: 'clean', 'ask')\n")
    if user_action.lower() == "clean":
        print(query_llm("Explain the process of cleaning data in Machine Learning."))
        cleaned_data = clean_data(data)
        print("Cleaned Data:\n", cleaned_data)
    elif user_action.lower() == "ask":
        question = input("Ask the LLM a question about the data or Machine Learning: ")
        print(query_llm(question))
        return

    print(query_llm("Explain the importance of training an ML model after data cleaning."))
    print("Training a Logistic Regression model...")
    model, accuracy, report = train_model(cleaned_data)
    print("Model trained! Accuracy: {:.2f}%".format(accuracy * 100))
    print("Classification Report:\n", report)

    print(query_llm(f"Explain the results of this model's accuracy ({accuracy:.2f}) and its classification report:\n{report}"))

if __name__ == "__main__":
    play_game()


In [ ]:
import torch
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from transformers import LlamaForCausalLM, LlamaTokenizer
import random
from scipy import stats

# Block 1: Dataset Generation
def generate_dataset():
    np.random.seed(42)
    data = pd.DataFrame({
        'Feature_A': np.random.rand(100) * 100,
        'Feature_B': np.random.rand(100) * 100 + 10,
        'Feature_C': np.random.rand(100) * 100 - 5,
        'Target': np.random.choice([0, 1], 100)
    })
    data['Feature_B'] += data['Feature_A'] * 0.8  # Correlation
    return data

# Block 2: PyTorch Model Definition and Training
class SimpleNN(torch.nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = torch.nn.Linear(input_dim, 64)
        self.layer2 = torch.nn.Linear(64, 32)
        self.output = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return torch.sigmoid(self.output(x))

def train_model(data, test_size=0.2, learning_rate=0.001, epsilon=1e-4):
    X = data.drop(columns=['Target']).values
    y = data['Target'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

    model = SimpleNN(X_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.BCELoss()

    # Training loop
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        if loss.item() < epsilon:
            break

    # Evaluate the model
    with torch.no_grad():
        output = model(X_test)
        predicted = (output > 0.5).float()
        accuracy = (predicted == y_test).float().mean().item()

    return accuracy

# Block 3: Outlier Detection
def detect_outliers(data):
    z_scores = np.abs(stats.zscore(data.drop(columns=['Target'])))
    outliers = np.where(z_scores > 3)  # Flag values with z-score > 3
    return outliers

# Block 4: RL Agent Actions
def agent_action(data, action, params):
    feedback = ""
    reward = 0  # Initialize reward

    if action == "remove_correlated":
        corr_matrix = data.corr()
        to_remove = [col for col in corr_matrix if corr_matrix['Feature_A'][col] > 0.8 and col != 'Feature_A']
        if to_remove:
            data = data.drop(columns=to_remove)
            feedback = f"Removed highly correlated features: {to_remove}."
            reward = 10  # Reward for reducing redundancy
    elif action == "apply_pca":
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data.drop(columns=['Target']))
        pca = PCA(n_components=2)
        reduced_data = pca.fit_transform(scaled_data)
        data = pd.DataFrame(reduced_data, columns=['PC1', 'PC2'])
        data['Target'] = data.index.map(data['Target'])
        feedback = "Applied PCA, reduced features to 2 principal components."
        reward = 5  # Reward for reducing dimensionality
    elif action == "adjust_split":
        params['test_size'] = random.uniform(0.1, 0.4)
        feedback = f"Adjusted test/train split to {params['test_size']:.2f} for better evaluation."
        reward = 3  # Small reward for exploration
    elif action == "tune_hyperparams":
        params['learning_rate'] = random.uniform(0.01, 10.0)
        params['epsilon'] = random.uniform(1e-6, 1e-2)
        feedback = f"Set learning rate to {params['learning_rate']:.2f} and epsilon to {params['epsilon']:.6f}."
        reward = 7  # Reward for experimenting with hyperparameters
    elif action == "include_outliers":
        # First, check if outliers exist
        outliers = detect_outliers(data)
        if len(outliers[0]) > 0:
            feedback = "Outliers detected, including them in the dataset."
            data = data.append(data.iloc[outliers[0]])
            reward = -5  # Adding outliers could introduce noise, so a penalty
        else:
            feedback = "No outliers detected, skipping inclusion."
            reward = 0  # No reward if no outliers found
    return data, feedback, params, reward

# Block 5: LLM Query (Using Hugging Face Llama)
def query_llm(prompt):
    model_name = "huggingface/llama-7b"  # Llama model with 7 billion parameters
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    model = LlamaForCausalLM.from_pretrained(model_name)

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Block 6: RL Loop
def rl_loop():
    data = generate_dataset()
    print("Initial Dataset:\n", data.head())

    actions = ["remove_correlated", "apply_pca", "adjust_split", "tune_hyperparams", "include_outliers"]
    params = {'test_size': 0.2, 'learning_rate': 0.001, 'epsilon': 1e-4}

    reward_history = []
    accuracy_history = []

    for step in range(10):  # Simulate 10 RL steps
        action = random.choice(actions)
        print(f"Step {step + 1}: Agent chose action: {action}")
        
        data, feedback, params, reward = agent_action(data, action, params)

        print("Action Feedback:", feedback)
        llm_response = query_llm(f"Explain the impact of this action: {feedback}")
        print("LLM Response:", llm_response)

        accuracy = train_model(data, test_size=params['test_size'], learning_rate=params['learning_rate'], epsilon=params['epsilon'])
        print(f"Model Accuracy after this action: {accuracy:.2f}")

        # Log rewards and accuracy
        reward_history.append(reward)
        accuracy_history.append(accuracy)

        print(f"Reward for this step: {reward}")
        print("-" * 50)

    # Visualization
    visualize_results(reward_history, accuracy_history)

# Block 7: Visualization
def visualize_results(reward_history, accuracy_history):
    steps = list(range(1, len(reward_history) + 1))

    plt.figure(figsize=(12, 6))

    # Reward History
    plt.subplot(1, 2, 1)
    plt.plot(steps, reward_history, marker='o', color='blue')
    plt.title("Reward History")
    plt.xlabel("Steps")
    plt.ylabel("Reward")
    plt.grid(True)

    # Accuracy History
    plt.subplot(1, 2, 2)
    plt.plot(steps, accuracy_history, marker='o', color='green')
    plt.title("Accuracy History")
    plt.xlabel("Steps")
    plt.ylabel("Accuracy")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Main Execution
if __name__ == "__main__":
    rl_loop()


Proof of Concept Paper: Leveraging RPG Mechanics and AI for Gamified Machine Learning Education
Abstract
Machine learning (ML) concepts, particularly in supervised learning, can be challenging to grasp for beginners. To overcome these barriers, we propose a novel approach that combines role-playing game (RPG) mechanics with interactive ML problem-solving, using reinforcement learning (RL) agents and large language models (LLMs) like Llama. In this framework, players assume the role of a "data scientist adventurer" navigating through a fictional world where they solve ML tasks and unlock story progression. The RL agents interact with dynamic datasets, optimize hyperparameters, clean noisy data, and select appropriate models. The LLM provides contextual feedback, enhancing the learning process by explaining decision-making processes. The paper outlines the design, methodology, and future research directions for this gamified learning environment.

1. Introduction
The need for innovative educational approaches to teach complex machine learning (ML) concepts is evident in the growing demand for data scientists. Traditional methods of teaching ML often present abstract, theoretical material that can be difficult for beginners to engage with. In contrast, the integration of gamified learning offers an interactive and immersive environment that enhances the learning experience. This paper introduces a proof-of-concept for a role-playing game (RPG) where players, acting as "data scientist adventurers," tackle ML tasks to progress through a storyline. By leveraging reinforcement learning (RL) agents and large language models (LLMs) such as Hugging Face's Llama, the game provides dynamic feedback and guidance on decision-making, model selection, and hyperparameter tuning.

The core idea is to gamify the process of solving supervised learning tasks, where players face challenges such as data cleaning, feature selection, and model optimization. As players progress through the game, they unlock new skills, tools, and datasets, learning how to apply ML techniques in a hands-on, immersive environment.

2. Problem Statement
Machine learning education is often abstract and inaccessible, particularly for beginners who struggle with theoretical concepts and mathematical foundations. Additionally, most learning platforms do not engage students in a way that connects theoretical knowledge with real-world applications. We aim to address these challenges by creating a gamified learning experience that allows players to learn ML through interactive problem-solving. The game helps players understand core ML concepts such as supervised learning, feature engineering, model evaluation, and hyperparameter tuning, while providing immediate feedback on their choices.

Furthermore, incorporating RL agents into the framework allows for experimentation with different approaches, making the process of learning dynamic and adaptive. The integration of LLMs such as Llama serves to guide the player, providing explanations and context for each decision.

3. Proposed Solution
The proposed solution is a gamified, narrative-driven RPG that teaches machine learning concepts through interactive problem-solving. In this game, players assume the role of different characters with distinct abilities:

The Data Alchemist: Specializes in transforming raw data into usable features through data preprocessing and feature engineering.
The Model Wizard: Focuses on selecting algorithms, tuning hyperparameters, and optimizing models for accuracy.
The Data Explorer: Excels at exploratory data analysis (EDA) and visualizing data to gain insights.
The Debugging Knight: Specializes in identifying issues in data pipelines, debugging models, and ensuring data quality.
These characters work together to solve quests based on real-world ML problems such as classification, regression, and data cleaning. The game's world is set in a fictional kingdom where datasets are represented as "ancient scrolls," which are corrupted (noisy data) or locked behind ML problems.

4. System Design
4.1 Game Mechanics
The game uses traditional RPG mechanics adapted to ML problem-solving. These include:

Levels and Quests: Players progress through levels by solving ML problems, such as predicting harvest yields using weather data or optimizing a recommendation system. Each quest presents a unique challenge, requiring players to apply different ML techniques.
Resource Management: Players manage resources such as "mana" or "energy" to perform computationally expensive tasks, such as running grid searches or training complex models.
Skill Trees: Players unlock new skills as they progress, including advanced techniques like ensemble methods, feature selection, and hyperparameter tuning.
Combat System: Instead of traditional combat, players face problem-solving challenges. NPCs (non-playable characters) present problems such as imbalanced datasets, and players must use tools like SMOTE (Synthetic Minority Over-sampling Technique) to resolve them.
4.2 Reinforcement Learning (RL) Framework
RL agents are tasked with exploring different approaches to solving ML problems, such as selecting optimizers, detecting outliers, and tuning hyperparameters. These agents experiment with strategies like choosing between Adam or SGD optimizers and deciding which features to include or exclude. The agent receives feedback based on the success of their choices in solving the quests. Additionally, the LLM provides dynamic, context-sensitive responses to explain the outcomes of decisions, helping the RL agent learn from past experiences.

Outlier Detection: The RL agent is trained to detect and decide whether to include or exclude outliers in the dataset, learning through feedback provided by the LLM.
Optimizer and Loss Function Selection: The RL agent explores different optimizer choices (e.g., Adam, RMSProp) and loss functions (e.g., binary cross-entropy, mean squared error), receiving rewards based on the accuracy of the models they generate.
4.3 LLM Integration
LLMs such as Llama are integrated into the game to provide explanations and insights into the choices made by the RL agents. After each decision, the LLM generates a natural language explanation of the consequences of the selected optimizer or loss function, helping players (or RL agents) understand the rationale behind their choices.

5. Evaluation
5.1 User Studies
To evaluate the effectiveness of the game as an educational tool, we will conduct user studies to measure learning outcomes, engagement, and motivation. Participants will engage with the game and complete various ML-related quests. Pre- and post-tests will be used to measure improvements in ML knowledge, while qualitative feedback will assess the overall game experience.

5.2 Benchmarking
We will benchmark the game’s effectiveness against other ML education methods, such as online courses and tutorials. Metrics such as completion time, accuracy, and player feedback will be compared to determine whether the game provides a more engaging and effective learning experience.

5.3 Qualitative Analysis
Player feedback will be collected to gauge the user experience. This feedback will inform future iterations of the game, helping to refine the gameplay mechanics, difficulty levels, and educational content.

6. Results
We will present data from the user studies, including improvements in learning outcomes (e.g., accuracy on ML tasks), user engagement metrics (e.g., time spent playing, quests completed), and qualitative feedback on the game's impact on players’ understanding of ML concepts.

7. Conclusion and Future Work
This paper presents a novel gamified approach to teaching machine learning, using RPG mechanics and LLMs to create an immersive and interactive learning environment. By integrating RL agents into the game, we offer a dynamic, adaptive framework for solving ML problems. Future work will focus on expanding the game to include unsupervised learning tasks, enhancing the adaptiveness of LLM responses, and improving game mechanics for deeper engagement.

8. Future Research Directions
As the field of reinforcement learning evolves, future work could explore the use of this gamified environment as a training tool for RL agents. By embedding causal relationships in the game’s tasks, agents could learn to make decisions based on cause-effect reasoning rather than simple correlation-based learning. This would open up new possibilities for advancing RL research and developing intelligent systems that better understand the causal structure of the world.

Additionally, the integration of counterfactual exploration and reward structuring based on causal inference could enhance the learning experience for both human players and RL agents. This research could contribute to advancing causal RL, a rapidly growing field in artificial intelligence.

9. Tools and Frameworks
The technical aspects of the game include:

LLMs: Llama model from Hugging Face for generating dynamic content and providing explanations.
ML Pipelines: Supervised learning tasks embedded in the game (e.g., classification, regression, feature selection).
Game Development: Python-based development using libraries such as PyGame or Unity for integration with ML models.

## Block 1: Environment with ML Tasks (Model Training & Hyperparameter Tuning)

In [ ]:
import openai
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# OpenAI API key
openai.api_key = "your-api-key-here"

class RPGEnvironmentWithMLTasks:
    def __init__(self, max_steps=10):
        self.state = None  # Current state
        self.step_count = 0  # Steps in the episode
        self.max_steps = max_steps  # Maximum steps per episode
        self.hidden_dependency = "clean missing values"  # Causal dependency
        self.action_log = []  # Track actions for causality evaluation
        self.dataset = self.generate_dataset()

    def generate_dataset(self):
        """
        Generate a simple dataset to simulate an ML task.
        """
        X = np.random.rand(100, 3)  # 100 samples, 3 features
        y = 2 * X[:, 0] + 3 * X[:, 1] - 4 * X[:, 2] + np.random.randn(100)  # Linear relation with noise
        return X, y

    def reset(self):
        """
        Reset the environment to start a new episode.
        """
        self.step_count = 0
        self.state = "You are a data scientist adventurer starting your journey. Your task is to train a model to predict outcomes. What will you do first?"
        self.action_log = []
        return self.state

    def step(self, action):
        """
        Execute an action, transition to the next state, and return reward.
        The action could be related to cleaning data, choosing a model, or tuning hyperparameters.
        """
        self.step_count += 1
        self.action_log.append(action)

        if action == "clean missing values":
            self.dataset = self.clean_data(self.dataset)
            llm_response = f"The dataset has been cleaned, and missing values have been handled."
            reward = 10  # Reward for cleaning the dataset
        elif action == "select a regression model":
            llm_response = f"A regression model has been selected."
            reward = 5  # Reward for model selection
        elif action == "tune hyperparameters":
            model = LinearRegression()
            # Simulate hyperparameter tuning (just random for now)
            best_params = self.tune_hyperparameters(model)
            llm_response = f"Hyperparameters have been tuned. Best model found with parameters {best_params}."
            reward = 15  # Reward for tuning hyperparameters
        else:
            llm_response = f"Invalid action."
            reward = -1  # Penalty for invalid actions

        # Simulate model evaluation
        if action == "evaluate model":
            model = self.train_model()
            performance = self.evaluate_model(model)
            llm_response += f" The model performance is {performance:.2f}."
            reward = performance * 10  # Reward based on model performance

        # Episode termination condition
        done = self.step_count >= self.max_steps or "end" in llm_response.lower()

        # Update state
        self.state = llm_response
        return self.state, reward, done

    def clean_data(self, dataset):
        """
        Simulate cleaning missing data by filling NaNs with the column mean.
        """
        X, y = dataset
        X[np.random.randint(0, 100, 10), np.random.randint(0, 3, 10)] = np.nan  # Introducing some missing values
        # Fill missing values with column mean
        for i in range(X.shape[1]):
            col_mean = np.nanmean(X[:, i])
            X[np.isnan(X[:, i]), i] = col_mean
        return X, y

    def tune_hyperparameters(self, model):
        """
        Simulate hyperparameter tuning.
        """
        # Simulating hyperparameter tuning (returning a mock value)
        return {"learning_rate": random.choice([0.01, 0.1, 0.2])}

    def train_model(self):
        """
        Train a model on the dataset.
        """
        X, y = self.dataset
        model = LinearRegression()
        model.fit(X, y)
        return model

    def evaluate_model(self, model):
        """
        Evaluate the model using RMSE (Root Mean Squared Error).
        """
        X, y = self.dataset
        predictions = model.predict(X)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        return 1 / (1 + rmse)  # Inverse of RMSE as a performance metric


## Block 2: RL Agent Implementation

In [ ]:
class SimpleAgent:
    def __init__(self, actions):
        self.actions = actions  # List of possible actions

    def choose_action(self, state):
        """
        Randomly choose an action (for now).
        """
        return np.random.choice(self.actions)

# Define possible actions related to ML tasks
actions = [
    "clean missing values",
    "select a regression model",
    "tune hyperparameters",
    "evaluate model"
]

# Initialize agent
agent = SimpleAgent(actions)


## Block 3: Running Episodes (Agent and Environment Interaction)

In [ ]:
# Initialize the environment
env = RPGEnvironmentWithMLTasks(max_steps=10)

# Run multiple episodes
num_episodes = 5
episode_rewards = []  # Track rewards per episode
causal_sequences = []  # Track action sequences for causal analysis

for episode in range(num_episodes):
    print(f"\n--- Starting Episode {episode + 1} ---")
    state = env.reset()
    done = False
    total_reward = 0
    action_sequence = []

    while not done:
        print(f"Current State: {state}")
        
        # Agent selects an action
        action = agent.choose_action(state)
        action_sequence.append(action)
        print(f"Agent chooses action: {action}")
        
        # Environment responds
        state, reward, done = env.step(action)
        total_reward += reward
        print(f"Reward: {reward}")

    # Track rewards and sequences
    episode_rewards.append(total_reward)
    causal_sequences.append(action_sequence)
    print(f"Episode {episode + 1} finished with Total Reward: {total_reward}")


## Acknowledgments Section (Example)
“The authors would like to acknowledge OpenAI's ChatGPT for its substantial contribution in conceptualizing and generating the initial prototype for the LLM-powered RPG environment and RL agent integration. ChatGPT provided foundational code and ideas that were refined and expanded upon by the authors.”